In [2]:
%pip install spacy
#  datasets

In [9]:
# %pip install datasets
%%python3 -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 18.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


2023-11-27 20:23:49.584320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 20:23:49.584577: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 20:23:49.584675: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 20:23:49.600138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 20:23:52.990497: W tensorflow/compiler/

In [11]:
from datasets import load_dataset
import nltk
import pickle
from nltk.stem.snowball import FrenchStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import spacy
# import fr_dep_news_trf

In [12]:
translation_dataset = load_dataset('Nicolas-BZRD/Parallel_Global_Voices_English_French',
split='train').to_pandas()

translation_dataset.head()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/342060 [00:00<?, ? examples/s]

,en,fr
0,Jamaica: “I am HIV”,Jamaïque : J’ai le VIH
1,"It's widely acknowledged, in the Caribbean and...","Il est largement reconnu, dans les Caraïbes et..."
2,"For this woman, however, photographed in the s...","Pour cette femme, cependant, photographiée dan..."
3,As Bacon writes on her blog:,Comme Bacon écrit sur son blog:
4,"“When I asked to take her picture, I suggested...",“Quand je lui ai demandé de la prendre en phot...


In [26]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Punctuation Removal

In [15]:
# PUNCTUATION REMOVAL
import string
string.punctuation

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#storing the puntuation free text
translation_dataset['en'] = translation_dataset['en'].apply(lambda x:remove_punctuation(x))
translation_dataset['fr'] = translation_dataset['fr'].apply(lambda x:remove_punctuation(x))
translation_dataset.head()


,en,fr
0,Jamaica “I am HIV”,Jamaïque J’ai le VIH
1,Its widely acknowledged in the Caribbean and e...,Il est largement reconnu dans les Caraïbes et ...
2,For this woman however photographed in the str...,Pour cette femme cependant photographiée dans ...
3,As Bacon writes on her blog,Comme Bacon écrit sur son blog
4,“When I asked to take her picture I suggested ...,“Quand je lui ai demandé de la prendre en phot...


# Lower Case / Tokenization

In [16]:
# change text to lower case
translation_dataset['en'] = translation_dataset['en'].apply(lambda x: x.lower())
translation_dataset['fr'] = translation_dataset['fr'].apply(lambda x: x.lower())

# tokenize text
translation_dataset['tokenized_en'] = translation_dataset.apply(lambda row: nltk.word_tokenize(row['en']), axis=1)
translation_dataset['tokenized_fr'] = translation_dataset.apply(lambda row: nltk.word_tokenize(row['fr']), axis=1)


# Stopword Removal

In [19]:
en_stopwords = nltk.corpus.stopwords.words('english')
fr_stopwords = nltk.corpus.stopwords.words('french')
# print(fr_stopwords[0:10])

# function to remove stopwords from tokenized text
def remove_stopwords(text, stopwords):
    output= [i for i in text if i not in stopwords]
    return output
#applying the function
translation_dataset['en'] = translation_dataset['tokenized_en'].apply(lambda x:remove_stopwords(x, en_stopwords))
translation_dataset['fr'] = translation_dataset['tokenized_fr'].apply(lambda x:remove_stopwords(x, fr_stopwords))
translation_dataset.head()


,en,fr,tokenized_en,tokenized_fr
0,"[jamaica, “, hiv, ”]","[jamaïque, ’, vih]","[jamaica, “, i, am, hiv, ”]","[jamaïque, j, ’, ai, le, vih]"
1,"[widely, acknowledged, caribbean, elsewhere, f...","[largement, reconnu, caraïbes, ailleurs, crain...","[its, widely, acknowledged, in, the, caribbean...","[il, est, largement, reconnu, dans, les, caraï..."
2,"[woman, however, photographed, streets, kingst...","[cette, femme, cependant, photographiée, rues,...","[for, this, woman, however, photographed, in, ...","[pour, cette, femme, cependant, photographiée,..."
3,"[bacon, writes, blog]","[comme, bacon, écrit, blog]","[as, bacon, writes, on, her, blog]","[comme, bacon, écrit, sur, son, blog]"
4,"[“, asked, take, picture, suggested, wouldnt, ...","[“, quand, demandé, prendre, photo, jai, propo...","[“, when, i, asked, to, take, her, picture, i,...","[“, quand, je, lui, ai, demandé, de, la, prend..."


# Stemming

In [20]:
#defining the object for stemming
porter_stemmer = PorterStemmer()
fr_stemmer = FrenchStemmer()

# function for stemming
def stemming(text, stemmer):
    stem_text = [stemmer.stem(word) for word in text]
    return stem_text
translation_dataset['en'] = translation_dataset['en'].apply(lambda x: stemming(x, porter_stemmer))
translation_dataset['fr'] = translation_dataset['fr'].apply(lambda x: stemming(x, fr_stemmer))
translation_dataset.head()

,en,fr,tokenized_en,tokenized_fr
0,"[jamaica, “, hiv, ”]","[jamaïqu, ’, vih]","[jamaica, “, i, am, hiv, ”]","[jamaïque, j, ’, ai, le, vih]"
1,"[wide, acknowledg, caribbean, elsewher, fear, ...","[larg, reconnu, caraïb, ailleur, craint, hont,...","[its, widely, acknowledged, in, the, caribbean...","[il, est, largement, reconnu, dans, les, caraï..."
2,"[woman, howev, photograph, street, kingston, j...","[cet, femm, cepend, photographi, ru, kingston,...","[for, this, woman, however, photographed, in, ...","[pour, cette, femme, cependant, photographiée,..."
3,"[bacon, write, blog]","[comm, bacon, écrit, blog]","[as, bacon, writes, on, her, blog]","[comme, bacon, écrit, sur, son, blog]"
4,"[“, ask, take, pictur, suggest, wouldnt, photo...","[“, quand, demand, prendr, photo, jai, propos,...","[“, when, i, asked, to, take, her, picture, i,...","[“, quand, je, lui, ai, demandé, de, la, prend..."


# Lemmatization

In [30]:
# lemmatize
lemmatizer_en = WordNetLemmatizer()
nlp = spacy.load('fr_core_news_md')

# doc = nlp(u"voudrais non animaux yeux dors couvre.")
# for token in doc:
#     print(token, token.lemma_)
#  function for lemmatization
def en_lemmatizer(text, lemmatizer):
    lemma_text = [lemmatizer.lemmatize(word) for word in text]
    return lemma_text

def fr_lemmatizer(text):
    # print(text)
    # doc = nlp(text)
    lemma_text = [nlp(word).lemma_ for word in text]
    return lemma_text

translation_dataset['en'] = translation_dataset['en'].apply(lambda x:en_lemmatizer(x, lemmatizer_en))
translation_dataset['fr'] = translation_dataset['fr'].apply(lambda x:fr_lemmatizer(x))
translation_dataset.head()


AttributeError: ignored

# Vectorization

## One hot encoding

In [ ]:
# one hot encoding: we loose hierachy
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(translation_dataset)
X.toarray()[0]
vectorizer.get_feature_names()[:5]

In [ ]:
%pip install zeugma

In [ ]:
# word_embedding
from zeugma import TextsToSequences

sequencer = TextsToSequences()
embedded_sequ = sequencer.fit_transform(translation_dataset)
embedded_sequ[0]

In [ ]:
# normalize data
from keras.preprocessing import sequence
max_len = 40

pad_sequ = sequence.pad_sequences(embedded_sequ, maxlen=max_len)

In [ ]:
# couche embeding
longueur_dict = max(list(map(lambda x: max(x), pad_sequ)))+1
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(longueur_dict, 8, input_length = max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# word2 vec embedding

tokenize_sentences = []

for i in range(len(translation_dataset)):
    tokenize_sentences.append(nltk.tokenize.word_tokenize(translation_dataset[i]))

from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model_W2V = Word2Vec(sentences=tokenize_sentences, size=100, window=5, min_count=1, workers=4)
model_W2V.train(tokenize_sentences, total_examples=len(tokenize_sentences), epochs=50)

tokenize_sentences[0][1]
model_W2V.similar_by_word(tokenize_sentences[0][1])[:5]

In [ ]:
# visualization
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

def display_closestwords_tsnescatterplot_perso(model, word):
    arr = np.empty((0,100), dtype='f')
    word_labels = [word]

    numb_sim_words = 5

    # get close words
    close_words = model.similar_by_word(word)[:numb_sim_words]

    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)

    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]

    # color for words
    color = ['red']
    for i in range(numb_sim_words):
        color.append('blue')

    # display scatter plot
    plt.scatter(x_coords, y_coords, c = color)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(1, 5), textcoords='offset points')
    plt.xlim(min(x_coords)-100, max(x_coords)+100)
    plt.ylim(min(y_coords)-100, max(y_coords)+100)
    plt.show()

    print("Word most similar to : "+word)
    print([sim_word[0] for sim_word in close_words])

cited from https://www.kaggle.com/code/kelde9/tutorial-preprocessing-nlp-english-french-text

In [ ]:
pickle.dump( translation_dataset_lower, open( "translation_dataset_lower.pkl", "wb" ) )
translation_dataset_lower = pickle.load( open( "translation_dataset_lower.pkl", "rb" ) )

In [ ]:
translation_dataset_lower.shape

(342060, 4)